In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from pyarrow.parquet import ParquetFile
import pyarrow as pa 
from sklearn.model_selection import train_test_split
from torchvision.transforms import ToTensor, Compose, Resize, Lambda

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
def get_img_list(df):
    images=[]
    for number in range(df.shape[0]):
        for idx, channels in enumerate(df['X_jets'][number]):
            for i, row in enumerate(channels):
                if i==0:
                    img = row
                else:
                    img = np.vstack([img, row])
            if idx==0:
                final_img = img
            else:
                final_img = np.dstack([final_img, img])
        images.append(final_img)
    return images

In [5]:
def ds_create(paths):
    for idx,path in enumerate(paths):
        run = ParquetFile(path) 
        req_rows = next(run.iter_batches(batch_size = 5000)) 
        df = pa.Table.from_batches([req_rows]).to_pandas() 
        images= get_img_list(df)
        X_dataset = images
        y_dataset = df['y']
        print(len(X_dataset), y_dataset.shape)
        X_train_dataset, X_test_dataset, y_train_dataset, y_test_dataset = train_test_split(X_dataset,y_dataset , random_state=42,test_size=0.2, shuffle=True)
        if idx==0:
            X_train = X_train_dataset
            y_train = y_train_dataset
            X_test = X_test_dataset
            y_test = y_test_dataset
        else:
            X_train = np.concatenate((X_train,X_train_dataset),axis=0)
            y_train = np.concatenate((y_train,y_train_dataset),axis=0)
            X_test = np.concatenate((X_test,X_test_dataset),axis=0)
            y_test = np.concatenate((y_test,y_test_dataset),axis=0)
    
    return X_train,y_train, X_test, y_test

In [6]:
path0 = '/kaggle/input/e2e-p2/QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet'
path1 = '/kaggle/input/e2e-p2/QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540.test.snappy.parquet'
path2 = '/kaggle/input/e2e-p2/QCDToGGQQ_IMGjet_RH1all_jet0_run2_n55494.test.snappy.parquet'

paths = [path0,path1,path2]
X_train,y_train, X_test, y_test = ds_create(paths)

5000 (5000,)
5000 (5000,)
5000 (5000,)


In [7]:
from torch.utils.data import Dataset

class ImageData(Dataset):
    def __init__(self, images,labels):
        self.images = images
        self.labels = labels
        self.transform=Compose([ToTensor(),Resize((32,32))])

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.transform(self.images[idx]), self.labels[idx]

In [8]:
batch_size = 64

train_dataloader = DataLoader(ImageData(X_train,y_train), batch_size=batch_size, shuffle=True,)
test_dataloader = DataLoader(ImageData(X_test,y_test), batch_size=batch_size, shuffle=True)

In [9]:
class VGG12(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(VGG12, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(self.in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            
        )

        self.adap_pool_layer = nn.AdaptiveAvgPool2d((7, 7))

        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=512*7*7, out_features=4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(in_features=4096, out_features=self.num_classes)
        )
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.adap_pool_layer(x)
        # flatten to prepare for the fully connected layers
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [10]:
channels = 3 
num_classes = 1
num_epochs = 25
batch_size = 64
learning_rate = 3e-4

# model = VGG12(channels, num_classes)
model = VGG12(channels,num_classes).to(device)


# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

total_step = len(train_dataloader)

In [11]:
model

VGG12(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU()
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU()
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), paddin

In [12]:
def evaluate(loader):
    model.eval()
    total_samples = 0
    correct_samples = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            pred = model(inputs.float())
            predicted_labels = (pred.sigmoid().round())
            correct_samples += (predicted_labels.squeeze(-1) == labels).sum().item()
            total_samples += labels.shape[0]
    return correct_samples / total_samples * 100

In [13]:
for epoch in range(num_epochs):
    epoch_loss = 0
    model.train()
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    acc = evaluate(test_dataloader)
    print(f'Epoch {epoch}: Loss = {epoch_loss:.4f}, Test Accuracy = {acc:.2f}%')

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that d

Epoch 0: Loss = 130.5154, Test Accuracy = 50.50%
Epoch 1: Loss = 130.1917, Test Accuracy = 49.50%
Epoch 2: Loss = 130.3629, Test Accuracy = 50.50%
Epoch 3: Loss = 130.3562, Test Accuracy = 50.50%
Epoch 4: Loss = 127.0353, Test Accuracy = 52.90%
Epoch 5: Loss = 124.4747, Test Accuracy = 60.93%
Epoch 6: Loss = 123.6765, Test Accuracy = 62.33%
Epoch 7: Loss = 121.9521, Test Accuracy = 63.37%
Epoch 8: Loss = 120.0363, Test Accuracy = 62.53%
Epoch 9: Loss = 118.6060, Test Accuracy = 64.30%
Epoch 10: Loss = 118.1799, Test Accuracy = 64.63%
Epoch 11: Loss = 116.6904, Test Accuracy = 63.93%
Epoch 12: Loss = 115.9402, Test Accuracy = 65.10%
Epoch 13: Loss = 115.6065, Test Accuracy = 64.97%
Epoch 14: Loss = 114.3663, Test Accuracy = 64.10%
Epoch 15: Loss = 112.8168, Test Accuracy = 65.53%
Epoch 16: Loss = 112.0833, Test Accuracy = 64.10%
Epoch 17: Loss = 110.9005, Test Accuracy = 64.53%
Epoch 18: Loss = 109.2873, Test Accuracy = 65.50%
Epoch 19: Loss = 107.5503, Test Accuracy = 64.67%
Epoch 20: 